In [1]:
import pandas as pd
import os
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np

In [2]:
def read_results(root_dir, experiment_name):
    score_results = pd.read_csv(os.path.join(root_dir, "results", f"{experiment_name}_results.csv"), index_col=None)
    for column in score_results.columns:
        if "score:" in column:
            score_name = column.split(":")[1]
            break
    return score_results, score_name

def table_mean_std(score_results, score, prob_types):
    score_results = score_results.sort_values(["model", "dataset", "eval_split", "n_shots", "output_prob_type"])
    score_results = score_results.groupby(["model", "dataset", "eval_split", "n_shots", "output_prob_type"]).agg({
        f"score:{score}": ["mean", "std"], 
    })
    if prob_types is not None:
        score_results = score_results.loc[(slice(None), slice(None), slice(None), slice(None), prob_types), :].sort_index(level=["model", "dataset", "eval_split", "n_shots"])
    return score_results

root_dir = "./"
experiment = "gpt2-xl_n100"
score_results, score = read_results(root_dir, experiment)
table_mean_std(score_results, score, prob_types=["original_probs", "content_free_['idk']"])

score:accuracy   
                                                                  mean   
model   dataset eval_split n_shots output_prob_type                      
gpt2-xl agnews  test       0       content_free_['idk']       0.294395  \
                                   original_probs             0.294316   
                           1       content_free_['idk']       0.313868   
                                   original_probs             0.312132   
        cb      test       0       content_free_['idk']       0.321429   
                                   original_probs             0.321429   
                           1       content_free_['idk']       0.435714   
                                   original_probs             0.442857   
                           4       content_free_['idk']       0.535714   
                                   original_probs             0.532143   
                           8       content_free_['idk']       0.514286   
                                   original_probs             0.517857   
        rte     test       0       content_free_['idk']       0.480866   
                                   original_probs             0.480866   
                           1       content_free_['idk']       0.535018   
                                   original_probs             0.540072   
                           4       content_free_['idk']       0.540072   
                                   original_probs             0.539350   
                           8       content_free_['idk']       0.528520   
                                   original_probs             0.528520   
        sst2    test       0       content_free_['idk']       0.539264   
                                   original_probs             0.539154   
                           1       content_free_['idk']       0.579681   
                                   original_probs             0.575508   
                           4       content_free_['idk']       0.570675   
                                   original_probs             0.558375   
                           8       content_free_['idk']       0.572323   
                                   original_probs             0.562109   
        trec    test       0       content_free_['idk']       0.232800   
                                   original_probs             0.230800   
                           1       content_free_['idk']       0.313600   
                                   original_probs             0.308800   
                           4       content_free_['idk']       0.391600   
                                   original_probs             0.393200   
                           8       content_free_['idk']       0.364800   
                                   original_probs             0.333600   

                                                                   
                                                              std  
model   dataset eval_split n_shots output_prob_type                
gpt2-xl agnews  test       0       content_free_['idk']  0.002840  
                                   original_probs        0.002883  
                           1       content_free_['idk']  0.035714  
                                   original_probs        0.035989  
        cb      test       0       content_free_['idk']  0.000000  
                                   original_probs        0.000000  
                           1       content_free_['idk']  0.080456  
                                   original_probs        0.068465  
                           4       content_free_['idk']  0.059225  
                                   original_probs        0.074059  
                           8       content_free_['idk']  0.078246  
                                   original_probs        0.093644  
        rte     test       0       content_free_['idk']  0.005932  
                                   original_probs        0.005932  
             